### Dataset source: 
* https://www.kaggle.com/c/fake-news/data?select=train.csv

### Data Description
train.csv: A full training dataset with the following attributes:

* id: unique id for a news article
* title: the title of a news article
* author: author of the news article
* text: the text of the article; could be incomplete
* label: a label that marks the article as potentially unreliable

1: unreliable
0: reliable

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

### Importing Dataset

In [2]:
dataset_train = pd.read_csv("train.csv")

### Showing Data

In [3]:
dataset_train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


### Getting info regarding Data

In [4]:
print("\n\n***********TRAIN************\n\n")
dataset_train.info()



***********TRAIN************


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [5]:
dataset_train.shape

(20800, 5)

### Finding null values

In [6]:
dataset_train.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
dataset_train['text'] = dataset_train['text'].str.strip()
dataset_train['text'] = dataset_train['text'].replace(r'^\s*$', np.nan, regex=True)

dataset_train.dropna(subset=['text'], inplace=True)

In [8]:
dataset_train.shape

(20684, 5)

### Checking duplicates

In [9]:
dataset_train.duplicated(subset=["text"]).value_counts()

False    20378
True       306
dtype: int64

### Droping duplicates

In [10]:
dataset_train = dataset_train.drop_duplicates(subset={"text"}, keep='first', inplace=False)

### Dropping irrelavent columns

In [11]:
dataset_train = dataset_train.drop(["id","title","author"],axis=1)
# Removed author as if I would have replaced author with truth ratio where truth ratio of each author saying truth wrt number 
# of times he/she has commented 
# But this would have made the set more biased wrt author as will include the ratio of those also who by mistake has commented 
#wrong news for the very first time

### Comparing the number of rows for fake and true news

In [12]:
dataset_train[dataset_train.label==0].shape, dataset_train[dataset_train.label==1].shape

((10387, 2), (9991, 2))

* as the shape/ number of rows are comparable hence the dataset is not baised

In [13]:
dataset_train.columns

Index(['text', 'label'], dtype='object')

### Finding keywords

In [14]:
def keyword_selector(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [15]:
dataset_train["text"] = dataset_train["text"].apply(keyword_selector)

# Models

In [16]:
x_train, x_test, y_train, y_test = train_test_split(dataset_train["text"],dataset_train["label"], test_size=0.25)

### Vectorizarion

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [18]:
from sklearn.linear_model import LogisticRegression

LogisticRegression = LogisticRegression()
LogisticRegression.fit(xv_train,y_train)

LogisticRegression()

In [19]:
pred_LogisticRegression=LogisticRegression.predict(xv_test)

In [20]:
LogisticRegression.score(xv_test, y_test)

0.9448478900883219

In [21]:
from sklearn.ensemble import GradientBoostingClassifier

GradientBoostingClassifier = GradientBoostingClassifier(random_state=0)
GradientBoostingClassifier.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [22]:
pred_GradientBoostingClassifier = GradientBoostingClassifier.predict(xv_test)
GradientBoostingClassifier.score(xv_test, y_test)

0.9281648675171738

### Comparing the 2 models

In [25]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print("**********GradientBoostingClassifier*********")

clf_report = pd.DataFrame(classification_report(y_test, pred_GradientBoostingClassifier, output_dict=True))
print("Train Result:\n")
print(f"Accuracy Score: {accuracy_score(y_test, pred_GradientBoostingClassifier) * 100:.2f}%")
print(f"Error rate: {1-accuracy_score(y_test, pred_GradientBoostingClassifier):.2f}") 
print(f"CLASSIFICATION REPORT:\n{clf_report}")
print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred_GradientBoostingClassifier)}\n")        

print("**********LogisticRegression**********")

clf_report = pd.DataFrame(classification_report(y_test, pred_LogisticRegression, output_dict=True))
print("Train Result:\n")
print(f"Accuracy Score: {accuracy_score(y_test, pred_LogisticRegression) * 100:.2f}%")
print(f"Error rate: {1-accuracy_score(y_test, pred_LogisticRegression):.2f}") 
print(f"CLASSIFICATION REPORT:\n{clf_report}")
print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred_LogisticRegression)}\n")        


**********GradientBoostingClassifier*********
Train Result:

Accuracy Score: 92.82%
Error rate: 0.07
CLASSIFICATION REPORT:
                     0            1  accuracy    macro avg  weighted avg
precision     0.941452     0.914726  0.928165     0.928089      0.928511
recall        0.917808     0.939197  0.928165     0.928503      0.928165
f1-score      0.929480     0.926800  0.928165     0.928140      0.928182
support    2628.000000  2467.000000  0.928165  5095.000000   5095.000000
Confusion Matrix: 
 [[2412  216]
 [ 150 2317]]

**********LogisticRegression**********
Train Result:

Accuracy Score: 94.48%
Error rate: 0.06
CLASSIFICATION REPORT:
                     0            1  accuracy    macro avg  weighted avg
precision     0.950826     0.938604  0.944848     0.944715      0.944908
recall        0.941781     0.948115  0.944848     0.944948      0.944848
f1-score      0.946282     0.943335  0.944848     0.944809      0.944855
support    2628.000000  2467.000000  0.944848  5095.00